# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from d2l import torch as d2l
from torch import nn
from sklearn.model_selection import train_test_split
import csv
import torch

import spacy
from tqdm import tqdm
import re
import time
import pickle

ModuleNotFoundError: No module named 'spacy'

In [ ]:
import os

# Check if running in a conda environment
if 'CONDA_DEFAULT_ENV' in os.environ:
    print("Running in a conda environment. Conda environment:", os.environ['CONDA_DEFAULT_ENV'])
else:
    print("Not running in a conda environment.")

# Check if running in a virtual environment
if 'VIRTUAL_ENV' in os.environ:
    print("Running in a virtual environment. Virtual environment:", os.environ['VIRTUAL_ENV'])
else:
    print("Not running in a virtual environment.")

# Load Dataset

In [ ]:
kaggle_data = pd.read_json('train.json')
kaggle_data.head()

In [ ]:
# Split the first 90% of the data as the train set
test_size = round(len(kaggle_data)*0.1) 
train_size = len(kaggle_data) - test_size

# train_set, test_set = train_test_split(kaggle_data, test_size=len(kaggle_data)-train_size, random_state = False)
train_set = kaggle_data.iloc[:train_size]
test_set = kaggle_data.iloc[train_size:]

# train_set
test_size

kaggle_data['full_text']

In [ ]:
essays = kaggle_data['full_text'].tolist()
tokens = kaggle_data['tokens'].tolist()

# Word Embedding

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
x = [“Nothing suits me like suit”] # Extract ELMo features 
embeddings = elmo(x, signature=”default”, as_dict=True)[“elmo”] 
embeddings.shape